<a href="https://colab.research.google.com/github/shahkushal38/Data-Science-Projects/blob/master/news_sentiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install keras sklearn 

In [ ]:
!pip install tensorflow

In [ ]:
!pip install tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached keras-2.10.0-py2.py3-none-any.whl (1.7 MB)
  Using cached tensorflow_estimator-2.10.0-py2.py3-none-any.whl (438 kB)
  Using cached flatbuffers-22.9.24-py2.py3-none-any.whl (26 kB)
  Using cached tensorboard-2.10.1-py3-none-any.whl (5.9 MB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.1

In [ ]:
import tensorflow
import keras
from tensorflow.python.keras.layers.recurrent import LSTM

In [ ]:
from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Embedding
from tensorflow.python.keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string

### Read data

In [ ]:
dir = '/content/drive/MyDrive/Colab Notebooks/Datasets/news_sentiments.csv'
dir_Kushal = '/content/drive/MyDrive/BDA /news_sentiments.csv'
data = pd.read_csv(dir_Kushal)

In [ ]:
data.head()

,Date,Title,URL,sentiment,confidence,Unnamed: 5
0,05/01/17,Eliminating shadow economy to have positive im...,http://economictimes.indiatimes.com/news/econo...,POSITIVE,0.996185,NaN
1,05/01/17,Two Chinese companies hit roadblock with India...,http://economictimes.indiatimes.com/news/econo...,NEGATIVE,-0.955493,NaN
2,05/01/17,SoftBank India Vision gets new $100,http://economictimes.indiatimes.com/small-biz/...,POSITIVE,0.595612,NaN
3,05/01/17,Nissan halts joint development of luxury cars ...,http://economictimes.indiatimes.com/news/inter...,NEGATIVE,-0.996672,NaN
4,05/01/17,Despite challenges Rajasthan continues to prog...,http://economictimes.indiatimes.com/news/polit...,POSITIVE,0.997388,NaN


In [ ]:
data.sentiment.unique()

array(['POSITIVE', 'NEGATIVE'], dtype=object)

In [ ]:
data.sentiment.value_counts()

NEGATIVE    108118
POSITIVE     92382
Name: sentiment, dtype: int64

In [ ]:
data.shape

(200500, 6)

In [ ]:
pos = []
neg = []
for l in data.sentiment:
    if l == "POSITIVE":
        pos.append(0)
        neg.append(1)
    elif l == "NEGATIVE":
        pos.append(1)
        neg.append(0)

In [ ]:
data['Pos']= pos
data['Neg']= neg

In [ ]:
data = data.drop(["Date", "URL", "confidence", "Unnamed: 5", "sentiment"], axis=1);

In [ ]:
data['Text'] = data['Title']

In [ ]:
data = data.drop(["Title"], axis=1)

In [ ]:
data.head()

,Pos,Neg,Text
0,0,1,Eliminating shadow economy to have positive im...
1,1,0,Two Chinese companies hit roadblock with India...
2,0,1,SoftBank India Vision gets new $100
3,1,0,Nissan halts joint development of luxury cars ...
4,0,1,Despite challenges Rajasthan continues to prog...


In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

### Clean data

In [ ]:
def remove_punct(text):
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', text)
    return text_nopunct

data['Text_Clean'] = data['Text'].apply(lambda x: remove_punct(x))

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk import word_tokenize, WordNetLemmatizer
tokens = [word_tokenize(sen) for sen in data.Text_Clean]

In [ ]:
word_tokenize('longer long')

['longer', 'long']

In [ ]:
tokens

[['Eliminating',
  'shadow',
  'economy',
  'to',
  'have',
  'positive',
  'impact',
  'on',
  'GDP',
  'Arun',
  'Jaitley'],
 ['Two',
  'Chinese',
  'companies',
  'hit',
  'roadblock',
  'with',
  'Indian',
  'investments'],
 ['SoftBank', 'India', 'Vision', 'gets', 'new', '100'],
 ['Nissan',
  'halts',
  'joint',
  'development',
  'of',
  'luxury',
  'cars',
  'with',
  'Daimler',
  'Sources'],
 ['Despite',
  'challenges',
  'Rajasthan',
  'continues',
  'to',
  'progress',
  'Vasundhara',
  'Raje'],
 ['India',
  'questions',
  'composition',
  'of',
  'UN',
  'panel',
  'to',
  'draft',
  'SDG',
  'report'],
 ['Wall',
  'St',
  'flat',
  'as',
  'banks',
  'discretionary',
  'stocks',
  'fall',
  'tech',
  'rises'],
 ['We',
  'are',
  'stock',
  'specific',
  'in',
  'pharma',
  'and',
  'auto',
  'not',
  'convinced',
  'on',
  'IT',
  'Sudip',
  'Bandyopadhyay',
  'Inditrade',
  'Capital'],
 ['Cautious',
  'on',
  'pharma',
  'time',
  'to',
  'get',
  'into',
  'IT',
  'Mahante

In [ ]:
def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
lower_tokens = [lower_token(token) for token in tokens] 

In [ ]:
lower_tokens

[['eliminating',
  'shadow',
  'economy',
  'to',
  'have',
  'positive',
  'impact',
  'on',
  'gdp',
  'arun',
  'jaitley'],
 ['two',
  'chinese',
  'companies',
  'hit',
  'roadblock',
  'with',
  'indian',
  'investments'],
 ['softbank', 'india', 'vision', 'gets', 'new', '100'],
 ['nissan',
  'halts',
  'joint',
  'development',
  'of',
  'luxury',
  'cars',
  'with',
  'daimler',
  'sources'],
 ['despite',
  'challenges',
  'rajasthan',
  'continues',
  'to',
  'progress',
  'vasundhara',
  'raje'],
 ['india',
  'questions',
  'composition',
  'of',
  'un',
  'panel',
  'to',
  'draft',
  'sdg',
  'report'],
 ['wall',
  'st',
  'flat',
  'as',
  'banks',
  'discretionary',
  'stocks',
  'fall',
  'tech',
  'rises'],
 ['we',
  'are',
  'stock',
  'specific',
  'in',
  'pharma',
  'and',
  'auto',
  'not',
  'convinced',
  'on',
  'it',
  'sudip',
  'bandyopadhyay',
  'inditrade',
  'capital'],
 ['cautious',
  'on',
  'pharma',
  'time',
  'to',
  'get',
  'into',
  'it',
  'mahante

In [ ]:
from nltk.corpus import stopwords
stoplist = stopwords.words('english')

In [ ]:
stoplist

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:
def remove_stop_words(tokens):
    return [word for word in tokens if word not in stoplist]

In [ ]:
filtered_words = [remove_stop_words(sen) for sen in lower_tokens] 

In [ ]:
result = [' '.join(sen) for sen in filtered_words]

In [ ]:
result

['eliminating shadow economy positive impact gdp arun jaitley',
 'two chinese companies hit roadblock indian investments',
 'softbank india vision gets new 100',
 'nissan halts joint development luxury cars daimler sources',
 'despite challenges rajasthan continues progress vasundhara raje',
 'india questions composition un panel draft sdg report',
 'wall st flat banks discretionary stocks fall tech rises',
 'stock specific pharma auto convinced sudip bandyopadhyay inditrade capital',
 'cautious pharma time get mahantesh sabarad sbi cap securities',
 'rupee gains ground ends 9 paise higher 67 96',
 'regulator sebi issues guidelines evaluate company boards',
 'budget session second phase likely march 9 april 13',
 'mapping market many psu stocks hit 52',
 'amazon sees 160 per cent growth seller base 2016',
 'despite sharekhan acqusition bnp paribas hold stake geojit',
 'tata motors castrol ink three',
 'note ban may temporarily slowdown economy president pranab mukherjee',
 'people comp

In [ ]:
data['Text_Final'] = result

In [ ]:
data['tokens'] = filtered_words

In [ ]:
data = data[['Text_Final', 'tokens', 'Pos', 'Neg']]

In [ ]:
data.head()

,Text_Final,tokens,Pos,Neg
0,eliminating shadow economy positive impact gdp...,"[eliminating, shadow, economy, positive, impac...",0,1
1,two chinese companies hit roadblock indian inv...,"[two, chinese, companies, hit, roadblock, indi...",1,0
2,softbank india vision gets new 100,"[softbank, india, vision, gets, new, 100]",0,1
3,nissan halts joint development luxury cars dai...,"[nissan, halts, joint, development, luxury, ca...",1,0
4,despite challenges rajasthan continues progres...,"[despite, challenges, rajasthan, continues, pr...",0,1


In [ ]:
data.size

802000

### Split data into test and train

In [ ]:
data_train, data_test = train_test_split(data, test_size=0.10, random_state=42)

In [ ]:
all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

1749983 words total, with a vocabulary size of 46154
Max sentence length is 45


In [ ]:
all_test_words = [word for tokens in data_test["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

194277 words total, with a vocabulary size of 18599
Max sentence length is 38


### Load Google News Word2Vec model

In [ ]:
import gensim.downloader as api
word2vec = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
# word2vec_path = 'GoogleNews-vectors-negative300.bin.gz'
# word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [ ]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

### Get Embeddings

In [ ]:
training_embeddings = get_word2vec_embeddings(word2vec, data_train, generate_missing=True)

In [ ]:
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 300

### Tokenize and Pad sequences

In [ ]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data_train["Text_Final"].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train["Text_Final"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

Found 46154 unique tokens.


In [ ]:
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

(46155, 300)


In [ ]:
test_sequences = tokenizer.texts_to_sequences(data_test["Text_Final"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

### Define RNN-LSTM

In [ ]:
label_names = ['Pos', 'Neg']

In [ ]:
y_train = data_train[label_names].values
print(y_train)

[[1 0]
 [0 1]
 [0 1]
 ...
 [1 0]
 [1 0]
 [0 1]]


In [ ]:
x_train = train_cnn_data
y_tr = y_train

In [ ]:
def recurrent_nn(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

#     lstm = LSTM(256, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)(embedded_sequences)
    lstm = LSTM(256)(embedded_sequences)
    
    x = Dense(128, activation='relu')(lstm)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [ ]:
model = recurrent_nn(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 
                len(list(label_names)))

  <tf.Variable 'lstm/lstm_cell/recurrent_kernel:0' shape=(256, 1024) dtype=float32>
  <tf.Variable 'lstm/lstm_cell/bias:0' shape=(1024,) dtype=float32>. This is a strong indication that the Lambda layer should be rewritten as a subclassed Layer.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 50, 300)      13846500    ['input_1[0][0]']                
                                                                                                  
 tf.compat.v1.shape (TFOpLambda  (3,)                0           ['embedding[0][0]']              
 )                                                                                                
                                                                                                  
 tf.__operators__.getitem (Slic  ()                  0           ['tf.compat.v1.shape[0][0]'] 

### Train RNN-LSTM

In [ ]:
num_epochs = 5
batch_size = 100

y = w * x 



In [ ]:
hist = model.fit(x_train, y_tr, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size)

Epoch 1/5
1625/1625 [==============================] - 78s 44ms/step - loss: 0.6153 - acc: 0.6565 - val_loss: 0.5978 - val_acc: 0.6699
Epoch 2/5
1625/1625 [==============================] - 70s 43ms/step - loss: 0.5904 - acc: 0.6812 - val_loss: 0.5862 - val_acc: 0.6814
Epoch 3/5
1625/1625 [==============================] - 73s 45ms/step - loss: 0.5809 - acc: 0.6901 - val_loss: 0.5848 - val_acc: 0.6810
Epoch 4/5
1625/1625 [==============================] - 70s 43ms/step - loss: 0.5745 - acc: 0.6948 - val_loss: 0.5765 - val_acc: 0.6881
Epoch 5/5
1625/1625 [==============================] - 72s 44ms/step - loss: 0.5682 - acc: 0.7009 - val_loss: 0.5737 - val_acc: 0.6903


### Test RNN-LSTM

In [ ]:
predictions = model.predict(test_cnn_data, batch_size=1024, verbose=1)

20/20 [==============================] - 1s 35ms/step


In [ ]:
labels = [1, 0]

In [ ]:
prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])

In [ ]:
# sum(data_test.Label==prediction_labels)/len(prediction_labels)
data_test["predicted"] = prediction_labels

In [ ]:
data_test.head()

,Text_Final,tokens,Pos,Neg,predicted
46855,uber picks new head indonesia business,"[uber, picks, new, head, indonesia, business]",0,1,0
178187,check health every day avoid mega hospital bills,"[check, health, every, day, avoid, mega, hospi...",1,0,1
126539,cbi summons west bengal minister partha chatte...,"[cbi, summons, west, bengal, minister, partha,...",1,0,1
124442,decelerating global economic growth attracting...,"[decelerating, global, economic, growth, attra...",0,1,0
189740,sebi hire agency depth risk assessment infra,"[sebi, hire, agency, depth, risk, assessment, ...",1,0,0


In [ ]:
N = np.array(data_test["Neg"])
P = np.array(data_test["Pos"])

In [ ]:
correct_prediction = 0
for i in range(len(prediction_labels)):
  if prediction_labels[i] == 1 and P[i] == 1:
    correct_prediction += 1
  if prediction_labels[i] == 0 and N[i] == 1:
    correct_prediction += 1
print(correct_prediction)

14088


In [ ]:
len(prediction_labels)

20050

In [ ]:
correct_prediction / len(prediction_labels)

0.702643391521197

Naive Bayes

In [ ]:
data.head()

In [ ]:
# x_train, x_test, y_train, y_test = train_test_split(data, test_size=0.10, random_state=42)

In [ ]:
data.head()

,Text_Final,tokens,Pos,Neg
0,eliminating shadow economy positive impact gdp...,"[eliminating, shadow, economy, positive, impac...",0,1
1,two chinese companies hit roadblock indian inv...,"[two, chinese, companies, hit, roadblock, indi...",1,0
2,softbank india vision gets new 100,"[softbank, india, vision, gets, new, 100]",0,1
3,nissan halts joint development luxury cars dai...,"[nissan, halts, joint, development, luxury, ca...",1,0
4,despite challenges rajasthan continues progres...,"[despite, challenges, rajasthan, continues, pr...",0,1


In [ ]:
# x_test = x_train
# y_test = y_train[:,0]
# print(y_train[:,0])

[1 0 0 ... 1 1 0]


In [ ]:
#Naive bayes code 

from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()

model.fit(x_train, y_train[:,0])

MultinomialNB()

In [ ]:
model.score(x_test, y_test)

0.5451094486007204

SVM (Support Vector Machine)

In [ ]:
import time
from sklearn import svm
from sklearn.metrics import classification_report
# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(x_train, y_train[:,0])
t1 = time.time()
prediction_linear = classifier_linear.predict(x_test)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1
# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))